In [1]:
from pathlib import Path
import json

def load_jsonl(path):
    with open(path, 'r') as f:
        return [json.loads(line) for line in f]
    

def load_json(path):
    with open(path, 'r') as f:
        return json.load(f)

def save_json(path, data):
    with open(path, 'w') as f:
        json.dump(data, f, indent=4)


def save_jsonl(path, data):
    with open(path, 'w') as f:
        for line in data:
            json.dump(line, f)
            f.write('\n')

# Process created json files for inference and evaluation

After running the vqa_json_creator.py script, we have json files containing multiple QA pairs for each image and consistent with the LLaVA-Med format. In this notebook, we will process these json files to create finalized json file for inference and evaluation.

## Add relative file path to the image ID

In [2]:
def add_relative_path(input_file, output_file, path_table_file):
    data = load_json(input_file)
    path_table = load_json(path_table_file)
    new_data = [{'image': path_table[i['image']], 
                 'id': i['id'], 
                 'conversations': list(i['conversations'])} for i in data]
    save_json(output_file, new_data)

# usage
# input and output json files
input_json = Path("input_json") / "llava_med_instruct_mimicvqa_test_expertmodel.json"
output_json = Path("output_json") / "llava_med_instruct_mimicvqa_test_expertmodel_path.json"
# json file mapping each image ID to its relative path
path_table_json = "mimic_cxr_relpath.json"
add_relative_path(input_json, output_json, path_table_json)

## Split QA pairs into separate json files based on question type

In [3]:
# extract QA pairs from an old QA list which are of a specific question type at image level
def select_conversation(old_list, qtype, qtable):
    new_list = []
    prefix = ''
    suffix = f'\n<image>'
    for i in range(0, int(len(old_list) / 2)):
        if i == 0:
            text = old_list[i * 2]['value']
            test_split = text.split('\n')
            if len(test_split) == 2:
                prefix = ''
            elif len(test_split) == 3:
                prefix = f"{test_split[0]}\n"
            else:
                raise ValueError(f"Invalid prefix: {text}")

        question = (old_list[i * 2]['value'].split('?')[0]).split('\n')[-1]
        if qtable[f'{question}?'] == qtype:
            # add expertise and image
            if len(new_list) == 0:
                new_item = old_list[i * 2]
                new_item['value'] = f"{prefix}{question}?{suffix}"
                new_list.append(new_item)
                new_list.append(old_list[i * 2 + 1])                
            else:
                new_list.append(old_list[i * 2])
                new_list.append(old_list[i * 2 + 1])
    return new_list

# extract QA pairs from a list of QA pairs which are of a specific question type for the whole json
def select_qa(input_file, output_file, question_table_file, question_type):
    data = load_json(input_file)
    qtable = load_json(question_table_file)
    new_data = []
    for i in data:
        new_conversations = select_conversation(i['conversations'], question_type, qtable)
        if len(new_conversations) > 0:
            new_data.append({'image': i['image'], 
                             'id': i['id'], 
                             'conversations': new_conversations})
    
    save_json(output_file, new_data)

# usage
question_types = ['abnormality', 'level', 'location', 'presence', 'type', 'view']
question_table_file = 'mimic_vqa_qtype.json'
root = Path("output_json")

for qtype in question_types:
    output_dir = root / qtype
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
    
    # with expert model
    input_file = root / 'llava_med_instruct_mimicvqa_test_expertmodel_path.json'
    output_file = output_dir / 'llava_med_instruct_mimicvqa_test_expert.json'
    select_qa(input_file, output_file, question_table_file, qtype)

These JSON files can now be used for first-round inference. However, during training, expert information is only included in the first question of each QA conversation round. Therefore, to generate the inference correctly, the following changes need to be made:

* If a question q is not the first question in a QA conversation, a second-round inference is required.
* In this second-round inference, the first question in that QA conversation, along with its predicted answer from the first-round inference and the expert information, should be inserted before the question q.

In [4]:
# convert first-round inference answers to a json
def create_dict_id_predictions(input_file):
    data = load_jsonl(input_file)
    res = {}
    for i in data:
        key = f"{i['question_id']}_{i['prompt']}"
        res[key] = i['text']
    return res

question_types = ['abnormality', 'presence', 'view', 'location', 'level', 'type']
predictions_root = Path("../predicted_vqa_json/ckpt_llava_med_all_mimic_expert_1run")
predictions = {}
for qtype in question_types:
    input_file = predictions_root / qtype / 'llava_med_expert_all_mimic_expert_run1.jsonl'
    predictions.update(create_dict_id_predictions(input_file))

save_json('predictions.json', predictions)

In [6]:
# Insert first-round inference (predictions) and expert information into each QA pair
def add_expert_prompt(image_id, old_list, qtype, qtable, predictions):
    new_list = []
    prefix = ''
    suffix = ''
    for i in range(0, int(len(old_list) / 2)):
        if i == 0:
            text = old_list[i * 2]['value']
            text_split = text.split('\n')
            # pred_key = f"{text.split('<image>')[0]}<patch_token_holder>\n<image>"
            # pred_answer = predictions[f"{image_id}_{pred_key}"]
            pred_answer = predictions[f"{image_id}_{text}"]
            if len(text_split) == 2:
                expert = ''
                question = text_split[0]
            elif len(text_split) == 3:
                expert = f"{text_split[0]}\n"
                question = text_split[1]
            else:
                raise ValueError(f"Invalid first question prompt: {text}")
            prefix = f"{expert}{question}\n<patch_token_holder>\n###{pred_answer}\n###Human: "
            # check question type 
            if qtable[f'{question}'] == qtype:
                new_item = old_list[i * 2]
                new_item['value'] = f"{expert}{question}\n<patch_token_holder>"
                new_list.append(new_item)
                new_list.append(old_list[i * 2 + 1])
        else:
            question = old_list[i * 2]['value']
            if qtable[f'{question}'] == qtype:
                # add expertise and image
                if len(new_list) == 0:
                    new_item = old_list[i * 2]
                    new_item['value'] = f"{prefix}{question}{suffix}"
                    new_list.append(new_item)
                    new_list.append(old_list[i * 2 + 1])                
                else:
                    new_list.append(old_list[i * 2])
                    new_list.append(old_list[i * 2 + 1])
    return new_list

# apply to the whole json file
def add_expert_prompt_json(input_file, output_file, question_table_file, question_type, predictions_file):
    data = load_json(input_file)
    qtable = load_json(question_table_file)
    predictions = load_json(predictions_file)
    new_data = []
    for i in data:
        new_conversations = add_expert_prompt(i['id'], i['conversations'], question_type, qtable, predictions)
        if len(new_conversations) > 0:
            new_data.append({'image': i['image'], 
                             'id': i['id'], 
                             'conversations': new_conversations})
    
    save_json(output_file, new_data)

# usage
question_types = ['abnormality', 'presence', 'view', 'location', 'level', 'type']
question_table_file = 'mimic_vqa_qtype.json'
predictions_file = 'predictions.json'
input_file = Path('output_json') / 'llava_med_instruct_mimicvqa_test_expertmodel_path.json'
for qtype in question_types:
    output_dir = Path('output_json') / qtype
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
    
    # with expert model
    output_file = output_dir / 'llava_med_instruct_mimicvqa_test_expert_2run.json'
    add_expert_prompt_json(input_file, output_file, question_table_file, qtype, predictions_file)

## Add question and answer type to testing json

In [7]:
def add_qa_type(input_file, output_file, question_type):
    data = load_json(input_file)
    new_data = []
    for i in data:
        question = (i['conversations'][0]['value'].split('?')[0]).split('\n')[-1]
        answer = i['conversations'][1]['value'].lower()
        if answer == 'yes' or answer == 'no':
            closed = 'closed'
        elif 'yes' in answer or 'no' in answer:
            closed = 'closed'
            print(f"Warning ambiguous question: {question} {answer}")
        else:
            closed = 'open'

        new_data.append({
            'image': i['image'], 
            'id': i['id'], 
            'conversations': i['conversations'],
            'question_type': question_type,
            'answer_type': closed 
        })

    save_json(output_file, new_data)

# usage
question_types = ['abnormality', 'level', 'location', 'presence', 'type', 'view']
root = Path('output_json')

for qtype in question_types:
    output_dir = root / qtype
    if not output_dir.exists():
        output_dir.mkdir(parents=True)
    
    # with expert model
    input_file = output_dir / 'llava_med_instruct_mimicvqa_test_expert.json'
    output_file = output_dir / 'llava_med_instruct_mimicvqa_test_expert_type.json'
    add_qa_type(input_file, output_file, qtype)